In [1]:
import pandas as pd
import glob
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from config import *

In [ ]:
file_path_cases = output_path+'/cases/*multianno.txt'
#list all txt files
cases = glob.glob(file_path_cases)

file_path_controls = output_path+'/control/*multianno.txt'
control = glob.glob(file_path_controls)

In [ ]:
genes =["ATG16L1", "NOD2", "PTPN22", "LACC1", "IL23R", "SLC39A8", "CARD9", "FUT2", "PTAFR", "HGFAC", "TAGAP", 
    "DOK2", "RELA", "IL10RA", "CCR7", "SDF2L1"] # Genes to consider
final_results = pd.DataFrame(columns=['patient_id'] + genes + ['label'])

for file in cases + control:
    df = pd.read_csv(file, sep="\t", low_memory=False)
    patient_id = file.split("/")[-1].replace(".hg19_multianno.txt", "")
    function_col = 'Func.refGene'
    genes_col = 'Gene.refGene'
    gt = df.columns[-1]
    # Apply lambda to assign variant counts: 2 for homozygous (1/1), 1 for heterozygous (1/0 or 0/1)
    df['variant_count'] = df[gt].apply(lambda x: 2 if x.startswith('1/1') else 1)
    # Group by gene and calculate the sum of variant counts
    gene_variant_counts = df.groupby(genes_col)['variant_count'].sum()
    variant_counts = gene_variant_counts.reindex(genes, fill_value=0)
    label = 1 if "cases" in file else 0
    result_row = pd.DataFrame([[patient_id] + variant_counts.tolist() + [label]], columns=['patient_id'] + genes + ['label'])
    final_results = pd.concat([final_results, result_row], ignore_index=True)
final_results = final_results.sample(frac=1, random_state=42).reset_index(drop=True)
final_results.to_csv(vcf_path + "/GenotypeCounts.csv", index=False)
print(final_results)


    patient_id ATG16L1 NOD2 PTPN22 LACC1 IL23R SLC39A8 CARD9 FUT2 PTAFR HGFAC  \
0         HC22       2    1      2     1     0       0     1    0     0     0   
1         CD19       1    0      2     1     0       0     2    1     0     0   
2         CD13       1    0      2     1     0       0     0    2     0     0   
3         HC28       1    0      2     2     0       0     2    0     0     0   
4          HC1       2    0      2     0     0       0     1    1     0     0   
..         ...     ...  ...    ...   ...   ...     ...   ...  ...   ...   ...   
106        HC5       2    0      2     0     0       0     1    0     0     0   
107       CD22       0    0      2     0     1       1     0    0     0     0   
108       HC35       0    0      1     1     0       0     1    1     0     0   
109       CD56       0    0      2     2     0       0     2    1     0     0   
110       HC44       0    0      2     1     0       1     1    1     0     1   

    TAGAP DOK2 RELA IL10RA 